In [1]:
import tensorflow as tf

In [2]:
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False
True


In [3]:
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features

In [14]:
dataset_path = 'data\mini_speech_commands'
for name in listdir(dataset_path):
    if isdir(join(dataset_path, name)):
        print(name)

Bom Kanan
Bom Kiri
Hidup
Jalan Kanan
Jalan Kiri
Lompat
Lompat Kanan
Lompat Kiri
Nunduk
Nunduk Tembak
Tembak
Tembak Atas
Tembak Kiri


In [15]:
all_target = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_target)

['Bom Kanan', 'Bom Kiri', 'Hidup', 'Jalan Kanan', 'Jalan Kiri', 'Lompat', 'Lompat Kanan', 'Lompat Kiri', 'Nunduk', 'Nunduk Tembak', 'Tembak', 'Tembak Atas', 'Tembak Kiri']


In [16]:
#To see how many files are in each
num_samples = 0
for target in all_target:
    print(len(listdir(join(dataset_path,name))));
    num_samples += len(listdir(join(dataset_path,name)))
print(f"Total sample: {num_samples}")

3
3
3
3
3
3
3
3
3
3
3
3
3
Total sample: 39


In [17]:
# Settings
target_list = all_target
feature_sets_file = 'all_target_mfcc_sets.npz'
perc_keep_samples = 1.0 # 1.0 keep all the samples
val_ratio = 0.1
test_ratio = 0.1
sample_rate = 16000
num_mfcc = 16
len_mfcc = 16

In [18]:
filenames = []
y = []
for index, target in enumerate(target_list):
    print(join(dataset_path,target))
    filenames.append(listdir(join(dataset_path,target)))
    y.append(np.ones(len(filenames[index])) * index)

data\mini_speech_commands\Bom Kanan
data\mini_speech_commands\Bom Kiri
data\mini_speech_commands\Hidup
data\mini_speech_commands\Jalan Kanan
data\mini_speech_commands\Jalan Kiri
data\mini_speech_commands\Lompat
data\mini_speech_commands\Lompat Kanan
data\mini_speech_commands\Lompat Kiri
data\mini_speech_commands\Nunduk
data\mini_speech_commands\Nunduk Tembak
data\mini_speech_commands\Tembak
data\mini_speech_commands\Tembak Atas
data\mini_speech_commands\Tembak Kiri


In [19]:
print(y)
for item in y:
    print(len(item))

[array([0., 0., 0.]), array([1., 1., 1.]), array([2., 2., 2.]), array([3., 3., 3.]), array([4., 4., 4.]), array([5., 5., 5.]), array([6., 6., 6.]), array([7., 7., 7.]), array([8., 8., 8.]), array([9., 9., 9.]), array([10., 10., 10.]), array([11., 11., 11.]), array([12., 12., 12.])]
3
3
3
3
3
3
3
3
3
3
3
3
3


In [20]:
filenames_y = list(zip(filenames, y))
random.shuffle(filenames_y)
filenames, y = zip(*filenames_y)

In [21]:
print(len(filenames))
filenames = filenames[:int(len(filenames)* perc_keep_samples)]
print(len(filenames))

13
13


In [22]:
val_set_size = int(len(filenames) * val_ratio)
test_set_size = int(len(filenames) * test_ratio)

In [23]:
filenames_val = filenames[:val_set_size]
filenames_test = filenames[val_set_size:(val_set_size + test_set_size)]
filenames_train = filenames[(val_set_size + test_set_size):]

In [24]:
y_orig_val = y[:val_set_size]
y_orig_test = y[val_set_size:(val_set_size + test_set_size)]
y_orig_train = y[(val_set_size + test_set_size):]

In [28]:
def calc_mfcc(path):
    #Load wavefiles
    signal , fs = librosa.load(path, sr=sample_rate)

    #Create MFCCs from soundclip
    mfccs = python_speech_features.base.mfcc(signal,
    samplerate=fs,winlen = 0.256, winstep = 0.050, numcep = num_mfcc, nfilt = 26,
    nfft = 2048, preemph = 0.0, ceplifter = 0, appendEnergy = False, winfunc = np.hanning)

    return mfcc.transpose()

In [29]:
# TEST: Construct test set by computing MFCC of each WAV file
prob_cnt = 0
x_test = []
y_test = []
for index, filename in enumerate(filenames_train):
    
    # Stop after 500
    if index >= 500:
        break
    
    # Create path from given filename and target item
    path = join(dataset_path, target_list[int(y_orig_train[index])], 
                filename)
    
    # Create MFCCs
    mfccs = calc_mfcc(path)
    
    if mfccs.shape[1] == len_mfcc:
        x_test.append(mfccs)
        y_test.append(y_orig_train[index])
    else:
        print('Dropped:', index, mfccs.shape)
        prob_cnt += 1

TypeError: only size-1 arrays can be converted to Python scalars